## Kymograph Cluster Code

In [ ]:
import matplotlib
import warnings

matplotlib.rcParams["figure.figsize"] = [20, 10]
warnings.filterwarnings(action="once")

In [ ]:
import trenchripper as tr

In [ ]:
input_file_prefix = (
    "/n/groups/paulsson/Daniel/Image_analysis_pipeline/tiff_extraction/test_out_4/fov_"
)
output_path = "/n/groups/paulsson/Daniel/Image_analysis_pipeline/kymographs_6"
all_channels = ["channel_BF", "channel_RFP"]

kymo = tr.kymograph.kychunker(
    input_file_prefix=input_file_prefix,
    output_path=output_path,
    all_channels=all_channels,
    trench_len_y=270,
    padding_y=20,
    trench_width_x=30,
    t_chunk=25,
    y_percentile=85,
    y_min_edge_dist=50,
    smoothing_kernel_y=(9, 1),
    triangle_nbins=50,
    triangle_scaling=1.0,
    top_orientation=0,
    x_percentile=85,
    background_kernel_x=(301, 1),
    smoothing_kernel_x=(9, 1),
    otsu_nbins=50,
    otsu_scaling=1.0,
)

In [ ]:
kymo_controller = tr.cluster.dask_controller(
    walltime="00:30:00", local=False, n_workers=30, memory="2GB"
)
kymo_controller.startdask()
kymo_controller.daskcluster.start_workers()

In [ ]:
kymo_controller.daskclient

In [ ]:
kymo_controller.mapfovs(kymo.generate_kymograph, 30)

In [ ]:
props = kymo_controller.daskclient.gather(
    kymo_controller.futures
)  # this will hang until all futures are done

In [ ]:
kymo_controller.shutdown()